## CSV to MARC XML Script
Use generate_xml() to run, check out readme for full instructions. Built by UTSC's Digital Scholarship Unit.

### How does it work?

This script uses the Python library Pymarc ([Link to documentation](https://pymarc.readthedocs.io/en/latest/)) to create MARC records. 

First, it loops through each row in the CSV, and for each row, **creates a dictionary** called *record_dict* which maps the column headers (which are the field tags) to the contents each item in the row (which are the contents of the subfields), as well as the indicators for the field. The contents are in the form of a list, since it must include both the subfield tag and the contents itself. It is refered to as the *subfield_array*. On first glance, it may seem a bit complicated, but it is a simply a dictionary in this form:

```
record_dict = {FieldTag1 : [Indicator1, Indicator2, Subfield1Tag, Subfield1Content, Subfield2Tag, Subfield2Content ....],
               FieldTag2 : [Indicator1, Indicator2, Subfield1Tag, Subfield1Content, Subfield2Tag, Subfield2Content ....],
               ...
               }
```
               
The record dict for simple-example.csv would be:
```
record_dict = {'100': ['a', 'Luke', 'b', 'Skywalker'],
               '400': ['a', '25', 'b', 'May'],
               '700': ['a', 'X wing']
               }
```
To complicate things, any Tamil content must also be added to the MARC, and therefore must be added to the dictionary as well. The content is transliterated using the **Open-Tamil** library.

After the script creates the *record_dict*, it loops through it and adds each field to the record, using Pymarc. Although it is a bit more complex in the script itself, the general idea of adding the dictionary content to the MARC looks something like this:
```
    for col_tag in record_dict:
        record.add_field(
            Field(
                tag = tag_name,
                indicators = record_dict[col_tag][:2],    # Indicators are first two positions
                subfields = record_dict[col_tag][2:]      # Subfield data is the rest of the array
            )
        )
```             
Furthermore, a **leader** and a **control field** are added. 

Finally, it is validated using the library **lxml** and saved as an XML.

In [776]:
# Import statements 
import sys
sys.path.append("/home/dsu/python_libs/open-tamil")

import csv
import tamil
import pandas as pd

from lxml import etree
from datetime import date
from pymarc import XMLWriter, Record, Field
from transliterate import azhagi, jaffna, combinational, UOM, ISO, itrans, algorithm

In [777]:
# Set up of ISO table
ISO_table = ISO.ReverseTransliteration.table

# Declare arrays to track number of record fields 
trans_field_list = []
def_field_list = []

In [778]:
# Function that runs validation on final XML file

def validate(xml_path: str, xsd_path: str) -> bool:
    """ Returns true if the xml at
    xml_path matches the schema at
    xsd_path. """
    
    xmlschema_doc = etree.parse(xsd_path)
    xmlschema = etree.XMLSchema(xmlschema_doc)
    xml_doc = etree.parse(xml_path)
    result = xmlschema.validate(xml_doc)
    
    return result

In [779]:
# Function that returns the code of a subfield, for example,
# given '100 $c | Main-entry" it returns 'c'. 

def get_subfield_code(title: str) -> str:
    """ Returns the letter code for a 
    subfield given the title of the csv
    column. Returns 'a' if no code is
    found. """
    
    # Find index of $
    i = title.rfind('$')
    
    # Return letter following $ if it exists,
    # otherwise return 'a'
    if i == -1:
        return 'a'
    else:
        return title[i+1]

In [780]:
# Function that returns the code of a field. For instance, if
# given '100 $c | Main-entry" it returns '100'. 

def get_tag(title: str) -> str:
    """ Returns the tag for a record
    given the title of the csv
    column. Returns 'None' if there
    is no title. """
    
    # Check if title exists
    if title == 'nan':
        # Return 'None' if none exists
        return "None"
    else:
        # If title is found, get indexes of
        # first space and slash
        i = title.find(' ')
        j = title.find('/')
        
        # If no space and no slash is found
        if i == -1 and j == -1:
            # Return the entire title
            return title
        # If no space and a slash is found
        elif i == -1 and j != -1:
            # Return value up to slash
            return title[:j]
        # Otherwise there is a space
        else:
            # Return tag up to space
            return title[:i]

In [781]:
# Function for Tamil transliteration

def get_transliterated(content: str) -> str:
    """ Returns the transliterated
    content string with ISO 15919. """
    
    # Use open-tamil algorithim to transliterate
    return algorithm.Direct.transliterate(ISO_table, content)

In [782]:
# This function gets the subfield array for a column.

def get_subfield_array(content: str, category: str, title: str, key: str) -> list:
    """ Returns an array with subfield codes and data,
    given the column category, title, key and cell
    content. Tamil content will be transliterated.
    If no content exists, or is '---', returns an 
    empty list."""
    
    subfield_array = []
    
    # Check if content exists
    if ('---' in content) or (content == 'nan'):
        # If it does not exist, return no subfields
        return subfield_array

    # Get subfield code
    sub_code = get_subfield_code(title)

    # Check if content is in Tamil
    if "(Tamil)" in category:
        # Find number of corresponding transliteration subfield
        if key in def_field_list or key == 'None':
            trans_count = len(def_field_list)
        else:
            def_field_list.append(key)
            trans_count = len(def_field_list)

        # Add a '0' if number is single digit
        if trans_count < 10: num = '0' + str(trans_count)
        else: num = str(trans_count)

        # Construct subfield array with Tamil specifications, including transliterated content
        subfield_array = ["6", "880-" + num, sub_code, get_transliterated(content)]
    else:
        # Construct subfield array without Tamil specifications
        subfield_array = [sub_code, content]
    
    return subfield_array

In [783]:
# Function returns a subfield array for a Tamil column

def get_trans_subfield_array(content: str, category: str, title: str, key: str) -> list:
    """ Returns an array with subfield codes
    and data, given the column category, title,
    key, and cell content for a Tamil column. 
    If no content exists, or is '---', returns 
    an empty list."""
    
    subfield_array = []
    
    # Check if content exists
    if ('---' in content) or (content == 'nan'):
        # If it does not exist, return no subfields
        return subfield_array
    
    # Get subfield code
    sub_code = get_subfield_code(title)

    if "(Tamil)" in category:
        # Find number of corresponding transliteration subfield
        if key in trans_field_list or key == 'None':
            trans_count = len(trans_field_list)
        else:
            trans_field_list.append(key)
            trans_count = len(trans_field_list)

        # Add a '0' if number is single digit
        if trans_count < 10: num = '0' + str(trans_count)
        else: num = str(trans_count)
            
        # Construct subfield array with Tamil specifications, including untransliterated content
        subfield_array = ["6", key + "-" + num, sub_code, content]
    else:
        # Construct subfield array without Tamil specifications (should never be called)
        subfield_array = [sub_code, content]
    
    return subfield_array

In [784]:
# Function handles getting string of 008 controlfield

def get_control_field(record_dict: dict) -> str:
    """ Given record_dict, returns a string
    for the control field 008. """
    
    data_str = ""
    
    # Check if date can be found
    if '260' in record_dict:
        if 'c' in record_dict['260']:            
            # Get date of publication, which follows the 'c' subfield in field 260
            i = record_dict['260'].index('c')
            pub_date = record_dict['260'][i+1]
            
            # Get the date entered (presumed the date created)
            today = str(date.today())
            year = today[:4]
            month = today[5:7]
            
            # Add to data string, as well as including language and type of publication
            data_str = year + month + "s" + pub_date + "    ii a          000 0 tam d"
    
    return data_str

In [785]:
def get_inds(category, csv_data) -> list:
    """ Returns a list of indicators
    for the category in csv_data. """
    
    ind_list = []
    data = csv_data[category][1]
    if data == data:
        ind_list = [data[0], data[2]]
        
    for i in range(len(ind_list)):
        if ind_list[i] == '_':
            ind_list[i] = ' '
    
    return ind_list

In [786]:
# Function responsible for returning the record_dict for each record

def get_record_dict(row, csv_data) -> dict:
    """Returns a dictionary mapping record tags
    to the subfield arrays, given a row of the 
    csv_data."""
    
    record_dict = {}
    
    # Loop through each column
    for category, col in csv_data.iteritems():         
        # Get title, content, and tag for each field
        title = str(col[0])
        content = str(row[category])
        raw_key = get_tag(title)
        inds = get_inds(category, csv_data)
        
        # Create subfield arrary
        sub_array = get_subfield_array(content, category, title, raw_key)

        # Check if field can be added to dictionary
        if raw_key != 'None' and content != 'nan' and content != '---':
            key = str(int(float(raw_key)))
            # Check if tag is already in dictionary
            if key in record_dict:
                # If tag is in dictionary, check if it has a Tamil component
                if "6" in record_dict[key] and len(sub_array) == 4:
                    # If it has a Tamil component, ignore the '6' subfield (already present, would be a duplicate)
                    record_dict[key] = record_dict[key] + sub_array[2:]
                else:
                    # If it does has a Tamil component, add entire subfield array
                    record_dict[key] = record_dict[key] + sub_array
            else:
                # If tag is not in dictionary, add subfield array to dictionary
                record_dict[key] = inds + sub_array
                
            if "(Tamil)" in category:
                sub_array = get_trans_subfield_array(content, category, title, key)
                trans_count = len(trans_field_list)
                # Check if tag exists
                if key != 'None' and len(sub_array) > 0:
                    # Check if tag is already in dictionary
                    if "880-" + str(trans_count) in record_dict:
                        # If tag is in dictionary, update subfield array
                        record_dict['880-' + str(trans_count)] = record_dict['880-' + str(trans_count)] + sub_array[2:]
                    else:
                        # If tag is not in dictionary, add subfield array to dictionary
                        record_dict['880-' + str(trans_count)] = inds + sub_array
                        
    return record_dict

In [790]:
def is_duplicate(category: str, tag: str):
    """ Returns true if category can be removed
    in order to avoid duplicate fields to abide
    by MARC rules."""
    
    # Check if it has been manually transliterated
    if '(English)' in category or 'General note' in category:
        # Check if in no-duplicate field/subfield
        blocked_fields = ['100', '245', '250', '500']
        num = get_tag(tag)
        if num in blocked_fields and 'a' in tag:
            return True
    
    return False

In [793]:
# Main function block

def generate_xml(outputname: str, inputname: str) -> None:
    """Generates MARC xml file from csv file with
    name inputname, using the pymarc library. Output
    is named with outputname. """
    
    csv_data = pd.read_csv(inputname)
    for category, col in csv_data.iteritems(): 
        if col[0] == col[0] and is_duplicate(category, col[0]):
            csv_data = csv_data.drop(category, 1)
    
    writer = XMLWriter(open(outputname + '.xml','wb'))
    
    # Loop through rows (records) of csv
    for i, row in csv_data.iterrows():
        # Skip first row (title row)
        if i >= 2:            
            # Clear out transliteration tracking arrays
            def_field_list.clear()
            trans_field_list.clear()
            
            # Create new record and get dictionaries
            leader_str = "00000cam a2200000Ma 4500"
            record = Record(leader=leader_str)
            record_dict = get_record_dict(row, csv_data)
            
            # Add 008 controlfield
            data_str = get_control_field(record_dict) 
            field = Field(tag='008', data=data_str)
            record.add_field(field)
            
            # Get indicators
            #ind_list = get_ind_list(csv_data)

            # Loop through tags in dictionary
            for col_tag in record_dict:
                if '880-' in col_tag: tag_name = '880'
                else: tag_name = col_tag
                # Add record to XML
                record.add_field(
                    Field(
                         tag = tag_name,
                         indicators = record_dict[col_tag][:2],
                         subfields = record_dict[col_tag][2:]
                    )
                )
           
            # Write record to XML
            writer.write(record)
    
    writer.close()

    # Validate output
    if validate(outputname + ".xml", "MARC21slim.xsd"):
        print("[MARC Record saved]")
    else:
        print('[WARNING: not valid.]')

In [794]:
# Run script on simple-example.csv with output files named 'OUTPUT'
generate_xml("OUTPUT", "TestRecords_MARCxml.csv")